In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.twit_module import twit_module
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on
model_name = 'model_test_05'
redownload=False


Here we download some training and validation data.

The training data is the semi-supervised Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
if redownload: 
    downloader = file_fetcher.downloader(using_notebook=True)
    # Validation data
    downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")
    
    # Compressed model
    # Note that this model has only been trained on 5% of the training data
    # Update when better-trained model is available
    downloader.download_file("https://www.dropbox.com/s/i88eqlja56xncyx/model_test_05.zip?dl=1","model_test_05.zip")
    
    # Extract all the contents of zip file in current directory
    with ZipFile(model_name + '.zip', 'r') as zipObj:
        zipObj.extractall()

In [4]:
# Load the validation data

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [5]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [6]:
# Executing this cell takes about 30s on a laptop

Classifier = twit_module.SentimentAnalyzer()
Classifier.load_models(model_name)

BoW model loaded successfully
LSTM model loaded successfully
Pre-trained embedding model loaded successfully


We santiy check the models: 

In [ ]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


In [18]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


array([1., 0., 1., 0.])

We test the model on an airline customer feedback dataset.

In [9]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(test_data['Labels'], predictions)))
sklearn.metrics.confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.785
Test MCC:  0.538


array([[6974, 2204],
       [ 273, 2090]], dtype=int64)

We have accuracy of just under 80%.

To improve our accuracy, we can refine the model on our airline data. The early stopping procedure (enabled by default to use 10% of the training data for validation) should prevent overfitting. We withold 20% for our own validation. 

In [10]:
trainX, testX, trainY, testY = sklearn.model_selection.train_test_split(test_data['Text'], test_data['Labels'])

In [11]:
Classifier.refine(trainX, trainY)
test_predictions = Classifier.predict(testX)


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 4s 498us/step - loss: 0.4143 - acc: 0.8264 - val_loss: 0.3403 - val_acc: 0.8568
Epoch 2/250
7789/7789 [==============================] - 3s 370us/step - loss: 0.3577 - acc: 0.8595 - val_loss: 0.3025 - val_acc: 0.8776
Epoch 3/250
7789/7789 [==============================] - 3s 375us/step - loss: 0.3196 - acc: 0.8796 - val_loss: 0.2820 - val_acc: 0.8811
Epoch 4/250
7789/7789 [==============================] - 3s 387us/step - loss: 0.3018 - acc: 0.8873 - val_loss: 0.2852 - val_acc: 0.8891
Epoch 5/250
7789/7789 [==============================] - 3s 371us/step - loss: 0.2981 - acc: 0.8955 - val_loss: 0.3008 - val_acc: 0.8834
Epoch 6/250
7789/7789 [==============================] - 3s 385us/step - loss: 0.3098 - acc: 0.8919 - val_loss: 0.2996 - val_acc: 0.8834
Epoch 7/250
7789/7789 [==============================] - 3s 399us/step - loss: 0.2970 - acc: 0.8943 - val_loss: 0.2873 - val_acc: 0

7789/7789 [==============================] - 3s 403us/step - loss: 0.2135 - acc: 0.9187 - val_loss: 0.2466 - val_acc: 0.9065
Epoch 61/250
7789/7789 [==============================] - 3s 425us/step - loss: 0.2205 - acc: 0.9182 - val_loss: 0.2467 - val_acc: 0.9065
Epoch 62/250
7789/7789 [==============================] - 3s 440us/step - loss: 0.2093 - acc: 0.9139 - val_loss: 0.2473 - val_acc: 0.9065
Epoch 63/250
7789/7789 [==============================] - 3s 425us/step - loss: 0.2171 - acc: 0.9149 - val_loss: 0.2485 - val_acc: 0.9065
Epoch 64/250
7789/7789 [==============================] - 3s 394us/step - loss: 0.2183 - acc: 0.9172 - val_loss: 0.2574 - val_acc: 0.9065
Epoch 65/250
7789/7789 [==============================] - 3s 383us/step - loss: 0.2175 - acc: 0.9199 - val_loss: 0.2572 - val_acc: 0.9065
Epoch 66/250
7789/7789 [==============================] - 3s 380us/step - loss: 0.2139 - acc: 0.9176 - val_loss: 0.2572 - val_acc: 0.9076
Epoch 67/250
7789/7789 [=======================

C:\Users\strix\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 7s 872us/step - loss: 1.0653 - acc: 0.7816 - val_loss: 0.7429 - val_acc: 0.8176
Epoch 2/250
7789/7789 [==============================] - 5s 705us/step - loss: 0.9035 - acc: 0.8049 - val_loss: 0.6763 - val_acc: 0.8268
Epoch 3/250
7789/7789 [==============================] - 5s 654us/step - loss: 0.8031 - acc: 0.8199 - val_loss: 0.6567 - val_acc: 0.8303
Epoch 4/250
7789/7789 [==============================] - 5s 623us/step - loss: 0.7269 - acc: 0.8299 - val_loss: 0.6093 - val_acc: 0.8441
Epoch 5/250
7789/7789 [==============================] - 5s 604us/step - loss: 0.6822 - acc: 0.8439 - val_loss: 0.5754 - val_acc: 0.8522
Epoch 6/250
7789/7789 [==============================] - 5s 585us/step - loss: 0.6096 - acc: 0.8482 - val_loss: 0.5351 - val_acc: 0.8545
Epoch 7/250
7789/7789 [==============================] - 4s 574us/step - loss: 0.5395 - acc: 0.8611 - val_loss: 0.5099 - val_acc: 0

7789/7789 [==============================] - 4s 530us/step - loss: 0.1905 - acc: 0.9424 - val_loss: 0.3841 - val_acc: 0.9053
Epoch 61/250
7789/7789 [==============================] - 4s 519us/step - loss: 0.1871 - acc: 0.9436 - val_loss: 0.3829 - val_acc: 0.9053
Epoch 62/250
7789/7789 [==============================] - 4s 516us/step - loss: 0.1768 - acc: 0.9457 - val_loss: 0.3820 - val_acc: 0.9076
Epoch 63/250
7789/7789 [==============================] - 4s 515us/step - loss: 0.1806 - acc: 0.9444 - val_loss: 0.3716 - val_acc: 0.9088
Epoch 64/250
7789/7789 [==============================] - 4s 509us/step - loss: 0.1819 - acc: 0.9451 - val_loss: 0.3695 - val_acc: 0.9088
Epoch 65/250
7789/7789 [==============================] - 4s 506us/step - loss: 0.1758 - acc: 0.9462 - val_loss: 0.3681 - val_acc: 0.9088
Epoch 66/250
7789/7789 [==============================] - 4s 498us/step - loss: 0.1739 - acc: 0.9484 - val_loss: 0.3670 - val_acc: 0.9111
Epoch 67/250
7789/7789 [=======================

7789/7789 [==============================] - 4s 533us/step - loss: 0.0962 - acc: 0.9711 - val_loss: 0.4053 - val_acc: 0.9145
Epoch 120/250
7789/7789 [==============================] - 4s 548us/step - loss: 0.1022 - acc: 0.9703 - val_loss: 0.4053 - val_acc: 0.9145
Epoch 121/250
7789/7789 [==============================] - 4s 524us/step - loss: 0.0988 - acc: 0.9694 - val_loss: 0.4053 - val_acc: 0.9157
Epoch 122/250
7789/7789 [==============================] - 4s 503us/step - loss: 0.0990 - acc: 0.9712 - val_loss: 0.4054 - val_acc: 0.9169
Epoch 123/250
7789/7789 [==============================] - 4s 495us/step - loss: 0.0934 - acc: 0.9714 - val_loss: 0.4055 - val_acc: 0.9169
Epoch 124/250
7789/7789 [==============================] - 4s 495us/step - loss: 0.0969 - acc: 0.9718 - val_loss: 0.4057 - val_acc: 0.9169
Epoch 125/250
7789/7789 [==============================] - 4s 498us/step - loss: 0.0952 - acc: 0.9709 - val_loss: 0.4059 - val_acc: 0.9169
Epoch 00125: early stopping


In [12]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, test_predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, test_predictions)))
sklearn.metrics.confusion_matrix(testY, test_predictions)

Test Accuracy:  0.913
Test MCC:  0.728


array([[2186,  113],
       [ 138,  449]], dtype=int64)

Now we have accuracies of over 90%! 

In [15]:
Classifier.evaluate(testX, testY)

BoW: 0.895
LSTM: 0.837
Pre-trained embedding: 0.889


{'ensembled': 0.913028413028413,
 'BoW': 0.8946638946638946,
 'LSTM': 0.8374913374913375,
 'Glove': 0.8891198891198892}